# Example Query for Survival Probability of 1-hop Queries

Queries our system in the form of:<br>
$P(survival\_time > X | Drug \wedge Disease)$<br>
Returned is a knowledge graph containing probability of survival time and genes that contributed strongly to the question of survival time w.r.t a drug and disease. It is our hope that these have some indication of gene sensitivites.

In [1]:
import requests
import json
import csv

# /predicate functionality example
By running /predicates you can extract a json object with the following predicates:<br>
1.) biolink:GeneToDiseaseAssociation<br>
2.) biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation<br>
3.) biolink:ChemicalToGeneAssociation<br>
4.) biolink:DiseaseToPhenotypicFeatureAssociation<br>

The above predicates match the following biolink entities:<br>
1.) biolink:Gene<br>
2.) biolink:Drug<br>
3.) biolink:Disease<br>
4.) biolink:PhenotypicFeature<br>

It should be noted for this handler that only the edge predicates biolink:GeneToDiseaseAssociation, biolink:ChemicalToGeneAssociation and biolink:DiseaseToPhenotypicFeatureAssociation are used to link a biolink:Gene and a biolink:Drug node to the biolink:Disease node and the biolink:Disease node to the biolink:PhenotypicFeature node.

In [2]:
r = requests.get('http://chp.thayer.dartmouth.edu/predicates/')
json_formatted_str = json.dumps(json.loads(r.content), indent=2)
print(json_formatted_str)

{
  "biolink:Gene": {
    "biolink:Disease": [
      "biolink:GeneToDiseaseAssociation"
    ]
  },
  "biolink:Drug": {
    "biolink:Disease": [
      "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation"
    ],
    "biolink:Gene": [
      "biolink:ChemicalToGeneAssociation"
    ]
  },
  "biolink:Disease": {
    "biolink:PhenotypicFeature": [
      "biolink:DiseaseToPhenotypicFeatureAssociation"
    ]
  }
}


# Build Query
Constructs a json query object and can take in a survival time, a disease and a drug.

In [3]:
# Function: buildQuery
#
# Input:
# -----------
# survival time, a disease and a drug
#
# Output:
# -----------
# A query graph that asks this probablistic question: 
# P(survival_time > X | Drug = d1 and Disease = Breast Cancer)

def buildQuery(st, disease, drug):
    
    # empty response
    reasoner_std = { "query_graph": dict(),
                     "knowledge_graph": dict(),
                     "results": list()
                   }
    # empty query graph
    reasoner_std["query_graph"] = { "edges": dict(),
                                    "nodes": dict()
                                  }
    # empty knowledge graph
    reasoner_std["knowledge_graph"] = { "edges": dict(),
                                        "nodes": dict()
                                      }
    
    node_count = 0
    edge_count = 0
    
    # wildcard gene slot
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'biolink:Gene',
                                                                     }
    node_count += 1
    
    # drug
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'biolink:Drug',
                                                                       'id':'{}'.format(drug[1])
                                                                     }
    node_count += 1
    
    # add in disease node
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'biolink:Disease',
                                                                       'id':'{}'.format(disease[1])
                                                                     }
    node_count += 1
    
    # link gene evidence to disease
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'biolink:GeneToDiseaseAssociation',
                                                                       'subject': 'n{}'.format(node_count -3),
                                                                       'object': 'n{}'.format(node_count -1)   # should be disease node
                                                                      }
    edge_count += 1
    
    # link drug evidence to disease
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation',
                                                                       'subject': 'n{}'.format(node_count -2),
                                                                       'object': 'n{}'.format(node_count -1)  # should be disease node
                                                                     }
    edge_count += 1
            
    # add target survival node
    phenotype = ('Survival_Time', 'EFO:0000714')
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category': 'biolink:PhenotypicFeature',
                                                                       'id': '{}'.format(phenotype[1]),
                                                                     }
    node_count += 1
    
    # link disease to target
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'biolink:DiseaseToPhenotypicFeatureAssociation',
                                                                       'subject': 'n{}'.format(node_count-2),
                                                                       'object': 'n{}'.format(node_count-1),
                                                                       'properties': { 'qualifier':'>=',
                                                                                       'days': st
                                                                                     }
                                                                     }
    return reasoner_std

# Read Drugs
Functionality to read in our set of available drugs with respective chembl curie IDs.

In [4]:
def readDrugs():
    with open('drug_curie_map.csv', 'r') as drug_file:
        reader = csv.reader(drug_file)
        next(reader)
        rows = [(row[0],row[1]) for row in reader]
    return rows

# Constructing the Query and pinging CHP
You can use the commented out functionality to check which drugs are available. Disease and drug are passed in as a tuple shown below. Currently only breast cancer can be used as the disease.

In [5]:
# list of drugs (and curies) we can query over
#drug_list = readDrugs()

survival_time = 1000
disease = ('Breast_Cancer', 'MONDO:0007254')
drug = ('CYCLOPHOSPHAMIDE', 'CHEMBL:CHEMBL88')

query = buildQuery(survival_time, disease, drug)
payload = {'message': query}

r = requests.post('http://chp.thayer.dartmouth.edu/query/', json=payload)
chp_res = json.loads(r.content)

# Extract sensitive genes
The very first result will be for the predicted survivability. Every result thereafter contains a gene with its respective sensitivity. Sensitivty values range between -1 and 1. Genes closer to -1 can be thought of as having contributed more to the false assignment of $P(survival\_time > X | Drug \wedge Disease)$. Similarly genes closer to 1 can be thought of as having contributed more to the true assignment. Gene sensitivities are order by their absolute value.

## 1. Extract predicted survivability

In [6]:
KG = chp_res['message']['knowledge_graph']
QG = chp_res['message']['query_graph']
results = chp_res['message']['results']

# holds probability of survival
survival_result = results[0]

for qge_id in survival_result['edge_bindings'].keys():
    if QG['edges'][qge_id]['predicate'] == 'biolink:DiseaseToPhenotypicFeatureAssociation':
        kge_id = survival_result['edge_bindings'][qge_id][0]['id']
        probability = KG['edges'][kge_id]['has_confidence_level']
        
print("P(survival_time > {} | drug & disease):".format(survival_time),probability)

P(survival_time > 1000 | drug & disease): 0.5310173697270472


## 2. Extract sensitive gene rankings

In [7]:
KG = chp_res['message']['knowledge_graph']
QG = chp_res['message']['query_graph']
results = chp_res['message']['results']

# holds gene sensitivites
sensitivity_results = results[1:]

genes = []
for sr in sensitivity_results:
    for qge_id in sr['edge_bindings'].keys():
        if QG['edges'][qge_id]['predicate'] == 'biolink:GeneToDiseaseAssociation':
            kge_id = sr['edge_bindings'][qge_id][0]['id']
            sensitivity = KG['edges'][kge_id]
            gene_curie = sensitivity['subject']
            gene_weight = sensitivity['value']    
    for qgn_id in sr['node_bindings'].keys():
        if QG['nodes'][qgn_id]['category'] == 'biolink:Gene':
            kgn_id = sr['node_bindings'][qgn_id][0]['id']
            gene_name = KG['nodes'][kgn_id]['name']
    genes.append((gene_name, gene_curie, gene_weight))
    
for gene in genes:
    print(gene)

('ENSEMBL:ENSG00000141510', 'TP53', -0.017925134747564825)
('ENSEMBL:ENSG00000129003', 'VPS13C', -0.01587301587301598)
('ENSEMBL:ENSG00000198626', 'RYR2', 0.015118923997428766)
('ENSEMBL:ENSG00000121879', 'PIK3CA', 0.015057113187954399)
('ENSEMBL:ENSG00000111642', 'CHD4', -0.013536567274885123)
('ENSEMBL:ENSG00000196632', 'WNK3', -0.013536567274885123)
('ENSEMBL:ENSG00000169855', 'ROBO1', -0.013227513227513315)
('ENSEMBL:ENSG00000118473', 'SGIP1', -0.013227513227513315)
('ENSEMBL:ENSG00000005810', 'MYCBP2', -0.013227513227513315)
('ENSEMBL:ENSG00000163554', 'SPTA1', 0.013091529446669718)
